In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import pickle
import warnings
import os 

warnings.filterwarnings('ignore')

class SVDRecommender:
    def __init__(self, n_factors=50):
        self.K = n_factors
        self.user_mapper = None
        self.app_mapper = None
        self.user_inv_mapper = None
        self.app_inv_mapper = None
        self.R = None  
        self.all_user_predicted_ratings = None  

    def _prepare_data(self, data_df):
        data_df['rating'] = np.log1p(data_df['hours']) 
        data_df = data_df[data_df['rating'] > 0]

        data_df['user_id'] = data_df['user_id'].astype(str)
        data_df['app_id'] = data_df['app_id'].astype(str)

        data_df['user_id'] = data_df['user_id'].astype('category')
        data_df['app_id'] = data_df['app_id'].astype('category')
        
        self.user_mapper = {user: idx for idx, user in enumerate(data_df['user_id'].cat.categories)}
        self.app_mapper = {app: idx for idx, app in enumerate(data_df['app_id'].cat.categories)}
        self.user_inv_mapper = {idx: user for user, idx in self.user_mapper.items()}
        self.app_inv_mapper = {idx: app for app, idx in self.app_mapper.items()}
        
        data_df['user_index'] = data_df['user_id'].cat.codes
        data_df['app_index'] = data_df['app_id'].cat.codes
        
        return data_df

    def fit(self, file_path):
        
        data_df = pd.read_csv(file_path)
        data_df = self._prepare_data(data_df)

        print(f"user amount: {len(self.user_mapper)}, game amount: {len(self.app_mapper)}")

        self.R = csr_matrix((
            data_df['rating'], 
            (data_df['user_index'], data_df['app_index'])
        ))
        
        try:
            U, sigma, Vt = svds(self.R, k=self.K) 
        except Exception as e:
            print(f"SVD running fail, error: {e}")
            return

        sigma = np.diag(sigma)
        self.all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
        
        print("model training completed.")

    def recommend(self, user_id, num_recommendations=10):
        
        if self.all_user_predicted_ratings is None:
            return "Error: model not been trained yet"
        
        try:
            user_index = self.user_mapper[user_id]
        except KeyError:
            return f"userID {user_id} is not included in training dataset."

        predicted_ratings_vector = self.all_user_predicted_ratings[user_index]
        
        played_games_indices = self.R[user_index, :].nonzero()[1]

        temp_ratings = predicted_ratings_vector.copy()
        temp_ratings[played_games_indices] = -np.inf

        top_game_indices = temp_ratings.argsort()[::-1][:num_recommendations]
 
        recommendations = []
        for app_index in top_game_indices:
            app_id = self.app_inv_mapper[app_index]
            predicted_score = predicted_ratings_vector[app_index]
            recommendations.append((app_id, predicted_score))
            
        return recommendations

    def save_model(self, filename='svd_recommender.pkl'):
        
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
        print(f"\nmodel has been saved to : {filename}")

def load_model(filename='svd_recommender.pkl'):
    if not os.path.exists(filename):
        print(f"Error: file {filename} not ex")
        return None
        
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    print(f"\nModel have been sucessfully loaded from {filename}.")
    return model


MODEL_FILENAME = 'game_svd_model.pkl'

recommender = SVDRecommender(n_factors=50)
recommender.fit('recommendations_small.csv') 
recommender.save_model(MODEL_FILENAME)

loaded_model = load_model(MODEL_FILENAME)

if loaded_model:
    temp_df = pd.read_csv('recommendations_small.csv')
    TEST_USER_ID = temp_df['user_id'].sample(1).iloc[0]

    print(f"\nRecommendations for user {TEST_USER_ID}")
    top_5_recs = loaded_model.recommend(TEST_USER_ID, num_recommendations=5)

    if isinstance(top_5_recs, str):
        print(top_5_recs)
    else:
        print("\nrecommendation result (app_id, predict rating):")
        for app_id, predicted_rating in top_5_recs:
            print(f"Game ID: {app_id:<10} | predicted rating: {predicted_rating:.4f}")

In [ ]:
loaded_model = load_model('game_svd_model.pkl')

In [ ]:
user_id_str = '7056396'  
recommendations = loaded_model.recommend(user_id_str)

print(f"\nRecommendation result for User {user_id_str}:")
if isinstance(recommendations, str):
    print(recommendations)
else:
    for app_id, predicted_rating in recommendations:
        print(f"Game ID: {app_id:<10} | predicted rating: {predicted_rating:.4f}")